In [1]:
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as pg
import plotly.express as px
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2
import dt4dds.analysis as analysis

data = analysis.GroupAnalysis([
    ('Genscript', analysis.ErrorAnalysis("../data/PCR/15c_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/PCR/20c_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/PCR/25c_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/PCR/10c_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/PCR/15c_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/PCR/20c_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/PCR/25c_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/0a_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/0b_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/2d_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/4d_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/7d_Genscript_GCall", paired=True, skip=[(102, 150), (102, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/0a_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/0b_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/2d_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/4d_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Genscript', analysis.ErrorAnalysis("../data/Aging/7d_Genscript_GCfix", paired=True, skip=[(117, 150), (117, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/15c_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/30c_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/45c_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/60c_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/75c_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/90c_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/15c_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/30c_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/45c_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/60c_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/75c_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/PCR/90c_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/0a_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/0b_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/2d_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/4d_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/7d_Twist_GCall", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/0a_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/0b_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/2d_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/4d_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
    ('Twist', analysis.ErrorAnalysis("../data/Aging/7d_Twist_GCfix", paired=True, skip=[(108, 150), (108, 150)])),
])

In [3]:
plot_data = data.data[f"substitutions_by_position"]
plot_data.sort_values(by="position")

fit_data = []
for read in plot_data['read'].unique():
    window = plot_data.loc[(plot_data['read'] == read)]
    medians = window.groupby('position', as_index=False)['rate'].median()
    rolled_medians = medians.rolling(10, min_periods=1, center=True, on="position").mean()
    fit_data.append(rolled_medians)

fig = pg.Figure()

colors = ['#2171b5', '#bdd7e7']
for i, iread in enumerate(plot_data.read.unique()):
    read_data = plot_data.loc[plot_data.read == iread]
    fig.add_trace(
        pg.Scatter(
            x=read_data['position'],
            y=read_data['rate']*1000,
            mode="markers",
            marker=dict(color=colors[i], size=2)
        )
    )


colors = ['#222222', '#222222']
for i, read_data in enumerate(fit_data):
    fig.add_trace(
        pg.Scatter(
            x=read_data.index,
            y=read_data['rate']*1000,
            mode="lines",
            line=dict(color=colors[i], width=1.5)
        )
    )

fig.update_layout(
    template="simple_white", 
    height=330, 
    width=330, 
    showlegend=False, 
    margin=dict(l=0, r=10, t=0, b=0),
    font_family="Inter",
    legend_font_size=28/3,
)
fig.update_xaxes(
    title_text='Position in read', 
    range=[0, 120], 
    dtick=20,
    title_font_family="Inter", 
    title_font_size=28/3, 
    tickfont_size=28/3, 
    minor_ticks="outside", 
    minor_dtick=10,
)
fig.update_yaxes(
    title_text='Substitution rate / 10<sup>-3</sup> nt<sup>-1</sup>', 
    range=[0, 5],
    dtick=1,
    title_font_family="Inter", 
    title_font_size=28/3, 
    tickfont_size=28/3, 
    minor_ticks="outside", 
    minor_dtick=0.5,
)
fig.show()
fig.write_image("error_rate_paired.svg")

In [4]:
for i, idata in enumerate(fit_data):
    display(idata.mean(), idata.std())
display(np.concatenate(fit_data).mean(), np.concatenate(fit_data).std())

rate    0.000543
dtype: float64

rate    0.000194
dtype: float64

rate    0.001154
dtype: float64

rate    0.00029
dtype: float64

0.0008484651872062558

0.0003919981345157489

In [5]:
plot_data = data.data[f"substitutions_by_type"]

def wavg(group):
    d = {}
    d['mean'] = group.ratio.mean()
    d['std'] = group.ratio.std()
    return pd.Series(d, index=['mean', 'std'])

df_aggregate = plot_data.groupby(['type', 'read'], as_index=False).apply(wavg)

fig = px.bar(
    df_aggregate, 
    x="type", 
    y="mean", 
    color="read",
    error_y="std",
    barmode='group',
    color_discrete_sequence=['#2171b5', '#bdd7e7']
)
for d in fig.data:
    d.error_y.color = "#222222"
    d.error_y.thickness = 1.5
    
fig.update_layout(
    template="simple_white", 
    height=330, 
    width=330, 
    showlegend=False, 
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Inter",
    legend_font_size=28/3,
)
fig.update_xaxes(
    title_text='Substitution pattern',
    categoryorder='array', 
    categoryarray= ['A2T', 'G2A', 'C2T', 'G2C', 'T2A', 'A2G', 'T2C', 'C2G', 'G2T', 'T2G', 'C2A', 'A2C'],
    title_font_family="Inter", 
    title_font_size=28/3, 
    tickfont_size=28/3, 
)
fig.update_yaxes(
    title_text='Fraction of substitutions', 
    tickformat=",.0%", 
    range=[0, 0.5],
    dtick=0.1,
    title_font_family="Inter", 
    title_font_size=28/3, 
    tickfont_size=28/3, 
    minor_ticks="outside", 
    minor_dtick=0.05,
)
fig.show()
fig.write_image("bias_paired.svg")